In [ ]:
import numpy as np
import pandas as pd
import requests
import keras
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import random
import calendar

%matplotlib inline

## MODELS FOR REPLAYING SEASON

In [ ]:
import random
class RandomAgent(object):

    def __init__(self):
        self.players = []
        self.agentId = random.random()
        self.cumulativeScore = 0

    def get_action(self, env):
        remaining_players = env.remaining_players
        index = random.randint(0,len(remaining_players) - 1)
        self.players.append(remaining_players[index])
        return index

    def addToScore(self, score):
        self.cumulativeScore = self.cumulativeScore + score
        
class PlayerStats(object):
    def __init__(self, REB = 0, AST = 0, STL = 0, BLK = 0, TO = 0, PTS = 0, THREES = 0, FTsA = 0, FGsM = 0, FTsM = 0, Gs = 0):
        self.REB = REB
        self.AST = AST
        self.STL = STL
        self.BLK = BLK
        self.TO = TO
        self.PTS = PTS
        self.THREES = THREES
        self.FTsA = FTsA
        self.FGsM = FGsM
        self.FTsM = FTsM
        self.Gs = Gs
        self.all = [
            self.REB, 
            self.AST, 
            self.STL,
            self.BLK, 
            self.TO, 
            self.PTS, 
            self.THREES, 
            self.FTsA, 
            self.FGsM, 
            self.FTsM, 
            self.Gs]
        
    def update(self, REB = 0, AST = 0, STL = 0, BLK = 0, TO = 0, PTS = 0, THREES = 0, FTsA = 0, FGsM = 0, FTsM = 0, Gs = 0):
        self.REB = self.REB+REB
        self.AST = self.AST+AST
        self.STL = self.STL+STL
        self.BLK = self.BLK+BLK
        self.TO = self.TO+TO
        self.PTS = self.PTS+PTS
        self.THREES = self.THREES+THREES
        self.FTsA = self.FTsA+FTsA
        self.FGsM = self.FGsM+FGsM
        self.FTsM = self.FTsM+FTsM
        self.Gs = self.Gs+Gs
        
class Draft(object):
    def __init__(self, agents, remaining_players, rounds):
        self.agents = agents
        self.remaining_players = remaining_players
        self.rounds = rounds
        self.current_round = 0
        self.current_agent_idx = 0
        self.num_agents = len(agents)
        
    def move(self):
        current_agent = self.agents[self.current_agent_idx]
        action = current_agent.get_action(self)
        self.remaining_players.pop(action)
        if(self.current_agent_idx == self.num_agents - 1):
            self.current_agent_idx = 0
            self.current_round += 1
        else:
            self.current_agent_idx += 1
                
    def is_draft_over(self):
        return self.current_round >= self.rounds 
    
    def runDraft(self):
        while(not self.is_draft_over()):
            self.move()
    
class MatchResult(object):
    def __init__(self):
        self.currentResult = (0,0)
        
    def addToP1(self): 
        self.currentResult = (self.currentResult[0] + 1, self.currentResult[1]) 
        
    def addToP2(self): 
        self.currentResult = (self.currentResult[0], self.currentResult[1] + 1)
    
    def caseP1Won(self, p1Won):
        if(p1Won):
            self.addToP1()
        else:
            self.addToP2()

def double_round_robin(units, sets = None):
    """ Generates a schedule of "fair" pairings from a list of units """
    count = len(units)
    sets = sets or (count - 1)
    half = count / 2
    for turn in range(sets):
        left = units[:half]
        right = units[count - half - 1 + 1:][::-1]
        pairings = zip(left, right)
        if turn % 2 == 1:
            pairings = [(y, x) for (x, y) in pairings]
        units.insert(1, units.pop())
        yield pairings
        
def comparePlayers(p1, p2):
    matchResult = MatchResult()
    matchResult.caseP1Won(p1.REB > p2.REB)
    matchResult.caseP1Won(p1.AST > p2.AST)
    matchResult.caseP1Won(p1.STL > p2.STL)
    matchResult.caseP1Won(p1.BLK > p2.BLK)
    matchResult.caseP1Won(p1.TO > p2.TO)
    matchResult.caseP1Won(p1.PTS > p2.PTS)
    matchResult.caseP1Won(p1.THREES > p2.THREES)
    matchResult.caseP1Won(p1.FTsA > p2.FTsA)
    matchResult.caseP1Won(p1.FGsM > p2.FGsM)
    matchResult.caseP1Won(p1.FTsM > p2.FTsM)
    return matchResult

def reducePlayerStats(previousStatus, currentStats):
    previousStatus.update(
                        REB=currentStats.REB, 
                        AST=currentStats.AST,
                        STL=currentStats.STL,
                        BLK=currentStats.BLK,
                        TO=currentStats.TO,
                        PTS=currentStats.PTS,
                        THREES=currentStats.THREES,
                        FTsA=currentStats.FTsA,
                        FGsM=currentStats.FGsM,
                        FTsM=currentStats.FTsM,
                        Gs=currentStats.Gs
                    )
    return previousStatus

In [ ]:
datetime.strptime('2014-10-30', '%Y-%m-%d').date()

## Formattting

In [ ]:
stats_2014_calc = pd.read_csv('stats_2014_player_stats_by_game')
from datetime import datetime, date
games_2014 = pd.DataFrame(pd.read_csv('games_2014.csv')).drop_duplicates('id')
columns = ['id', 'date']
dates = pd.DataFrame(games_2014[columns], columns=columns)
games_2014_dates = pd.merge(stats_2014_calc, dates, on='id')
dates_formatted = [datetime.strptime(x, '%Y-%m-%d').date() for x in games_2014_dates['date']]
games_2014_dates['date'] = dates_formatted

In [ ]:
#random

#def convert(x):
#    try:
#        return float(x)
 #   except Exception as e:
#        return 0.0
    
#modified = games_2014_dates.drop([14204, 14208, 24541])
#ts = modified['3PT']
#t = np.array([[convert(x) for x in threes.split('-')] for threes in ts])
#modified['3PTsMade'] = t.T[0]
#modified['3PTsAttempted'] = t.T[1]
#ts = modified['FT']
#t = np.array([[convert(x) for x in threes.split('-')] for threes in ts])
#modified['FTsMade'] = t.T[0]
#modified['FTsAttempted'] = t.T[1]
#ts = modified['FG']
#t = np.array([[convert(x) for x in threes.split('-')] for threes in ts])
#modified['FGsMade'] = t.T[0]
#modified['FGsAttempted'] = t.T[1]
#stats_2014 = modified

In [ ]:
cal = calendar.Calendar()
year_2014 = [cal.monthdatescalendar(2014, x) for x in range(1, 3)]
[item for sublist in year_2014 for item in sublist]

## RUN THE ENTIRE THING FOR 2014

In [122]:
players_2014 = list(set(games_2014_dates['player']))
draft = Draft([RandomAgent() for _ in range(10)], players_2014, 10)
draft.runDraft()

pairings = double_round_robin(draft.agents, sets = len(draft.agents) * 2 - 2)

cal = calendar.Calendar()
year_2014 = [cal.monthdatescalendar(2014, x) for x in range(1, 4)]
weeks = [item for sublist in year_2014 for item in sublist]
for (week, weekPairings) in zip(weeks, pairings): 
    week_stats = {}
    for day in week:
        games_today = games_2014_dates.loc[games_2014_dates['date'] == day].drop_duplicates('player')
        for _id, line in games_today.iterrows():
            if(week_stats.get(line.player) == None):
                week_stats[line.player] = PlayerStats(
                        REB=float(line.get('REB')), 
                        AST=float(line.get('AST')),
                        STL=float(line.get('STL')),
                        BLK=float(line.get('BLK')),
                        TO=float(line.get('TO')),
                        PTS=float(line.get('PTS')),
                        THREES=float(line.get('3PTsMade')),
                        FGsM=float(line.get('FGsMade')),
                        FTsM=float(line.get('FTsMade')),
                        FTsA=float(line.get('FTsAttempted')),
                        Gs=1
                    )
            else:
                player = week_stats.get(line.player)
                player.update(
                        REB=float(line.get('REB')), 
                        AST=float(line.get('AST')),
                        STL=float(line.get('STL')),
                        BLK=float(line.get('BLK')),
                        TO=float(line.get('TO')),
                        PTS=float(line.get('PTS')),
                        THREES=float(line.get('3PTsMade')),
                        FGsM=float(line.get('FGsMade')),
                        FTsM=float(line.get('FTsMade')),
                        FTsA=float(line.get('FTsAttempted')),
                        Gs=1
                    )
                week_stats[line.player] = player
    for (player1, player2) in weekPairings:
        player1Totals = []
        player2Totals = []
        for player in player1.players:
            #This player didn't play this week. So add empty stats
            if (week_stats.get(player) == None):
                 player1Totals.append(PlayerStats())
            else:
                player1Totals.append(week_stats.get(player))
        for player in player2.players:
            if (week_stats.get(player) == None):
                 player2Totals.append(PlayerStats())
            else:
                player2Totals.append(week_stats.get(player))
        player1Reduced = reduce( reducePlayerStats, player1Totals)
        player2Reduced = reduce( reducePlayerStats, player2Totals)
        p1Score, p2Score = comparePlayers(player1Reduced, player2Reduced).currentResult
        print(p1Score, p2Score)
        player1.addToScore(p1Score)
        player2.addToScore(p2Score)
draft

(3, 7)
(7, 3)
(3, 7)
(1, 9)
(6, 4)
(2, 8)
(0, 10)
(10, 0)
(8, 2)
(10, 0)
(3, 7)
(5, 5)
(1, 9)
(0, 10)
(8, 2)
(6, 4)
(5, 5)
(8, 2)
(3, 7)
(3, 7)
(1, 9)
(2, 8)
(7, 3)
(8, 2)
(5, 5)
(9, 1)
(0, 10)
(2, 8)
(8, 2)
(6, 4)
(10, 0)
(10, 0)
(7, 3)
(2, 8)
(7, 3)
(2, 8)
(7, 3)
(3, 7)
(0, 10)
(6, 4)
(1, 9)
(4, 6)
(1, 9)
(10, 0)
(6, 4)
(10, 0)
(1, 9)
(1, 9)
(9, 1)
(3, 7)
(1, 9)
(10, 0)
(8, 2)
(4, 6)
(0, 10)
(10, 0)
(4, 6)
(9, 1)
(10, 0)
(4, 6)
(0, 10)
(2, 8)
(2, 8)
(6, 4)
(7, 3)
(8, 2)
(8, 2)
(5, 5)
(3, 7)
(10, 0)
(0, 10)
(10, 0)
(3, 7)
(0, 10)
(3, 7)
(4, 6)
(1, 9)
(5, 5)
(6, 4)
(9, 1)


In [ ]:
c